# Using Prompts, Features as Tools and Knowledge 

By combining the Tecton GenAI functions, you can provide much better context in for your LLM applications.
- The *knowledge* function provides RAG functionality by taking unstructured text and calculating embedding vectors for them and storing them in a vector database that provides similarity search capabilities to better respond to user queries. 
- The *prompt* provides instructions for the LLM and is enriched with context from feature pipelines or directly from the real-time context.
- *Features as tools* give the LLM the ability to retrieve additional data when it needs it to respond to the user question.


<a target="_blank" href="https://colab.research.google.com/github/tecton-ai/gen-ai/blob/main/knowledge.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" width="150"/>
</a>

In this notebook, you will combine all three tools to create an example restaurant recommendation chatbot.

## Install Packages

In [1]:
_ = !pip install 'tecton-gen-ai[tecton,langchain,llama-index,dev]' langchain-openai llama-index-llms-openai

## Setup

In [3]:
from tecton_gen_ai.testing import set_dev_mode

set_dev_mode()

In this example you will use an OpenAI gpt-4o LLM model to test. 

You will need to provide an Open AI API key for this purpose.

Replace "your-openai-key" with your own key.

In [82]:
import os

# replace with your key
os.environ["OPENAI_API_KEY"] = "your-openai-key"

# Feature Views
In the following cell you create two mock feature views:
- user_info_fv: provides each user's profile information, including name, age and food_preference. 
- recent_eats_fv: provides a list of the last three restaurants each user visited.

In [6]:
from tecton_gen_ai.testing import make_local_batch_feature_view


# mock user info data
user_info = [
    {"user_id": "user1", "name": "Jim",  "age": 30, "food_preference": "American"},
    {"user_id": "user2", "name": "John", "age": 40, "food_preference": "Italian"},
    {"user_id": "user3", "name": "Jane", "age": 50, "food_preference": "Chinese"},
]
user_info_fv = make_local_batch_feature_view(
    "user_info_fv",
    user_info,
    ["user_id"],
    description="User's basic information."
)

# mock user's recent visits 
recent_eats = [
    {"user_id": "user1", "last_3_visits":str(["Mama Ricotta's", "The Capital Grille", "Firebirds Wood Fired Grill"])},
    {"user_id": "user2", "last_3_visits":str(["Mama Ricotta's", "Villa Antonio", "Viva Chicken"])},
    {"user_id": "user3", "last_3_visits":str(["Wan Fu", "Wan Fu Quality Chinese Cuisine", "Ru San's"])},
]
recent_eats_fv = make_local_batch_feature_view(
    "recent_eats_fv",
    recent_eats,
    ["user_id"],
    description="User's recent restaurant visits."
)

# Restaurant Descriptions - Source of Knowledge
You'll use a sample of restaurants descriptions as a base of knowledge. 

In [7]:
# create mock restaurant description data for knowledge
restaurant_descriptions = [
  {
    "name": "Mama Ricotta's",
    "description": "A Charlotte staple since 1992, Mama Ricotta's offers authentic Italian cuisine in a warm, family-friendly atmosphere. Known for their hand-tossed pizzas, homemade pasta, and signature chicken parmesan."
  },
  {
    "name": "The Capital Grille",
    "description": "An upscale steakhouse chain featuring dry-aged steaks, fresh seafood, and an extensive wine list. The Charlotte location offers a refined dining experience with impeccable service and a sophisticated ambiance."
  },
  {
    "name": "Firebirds Wood Fired Grill",
    "description": "A polished casual restaurant known for its classic American cuisine cooked over an open wood fire. Specialties include hand-cut steaks, fresh seafood, and signature cocktails in a warm, contemporary setting."
  },
  {
    "name": "Villa Antonio",
    "description": "An elegant Italian restaurant offering a romantic atmosphere and authentic cuisine. Known for its homemade pasta, extensive wine selection, and attentive service, perfect for special occasions."
  },
  {
    "name": "Viva Chicken",
    "description": "A fast-casual eatery specializing in Peruvian-style rotisserie chicken. Offers fresh, flavorful dishes with a modern twist on traditional Peruvian cuisine, including quinoa stuffed avocados and yuca fries."
  },
  {
    "name": "Wan Fu",
    "description": "A local favorite for Chinese cuisine, Wan Fu offers a wide array of traditional and Americanized Chinese dishes. Known for its generous portions, friendly service, and comfortable dining atmosphere."
  },
  {
    "name": "Wan Fu Quality Chinese Cuisine",
    "description": "An upscale Chinese restaurant focusing on authentic flavors and high-quality ingredients. Offers a more refined dining experience with a menu featuring both classic and innovative Chinese dishes."
  },
  {
    "name": "Ru San's",
    "description": "A popular sushi restaurant known for its extensive menu and all-you-can-eat option. Offers a wide variety of sushi rolls, sashimi, and other Japanese dishes in a casual, vibrant atmosphere."
  },
  {
    "name": "Le Bernardin",
    "description": "A world-renowned, Michelin three-star restaurant specializing in exquisite seafood. Chef Eric Ripert's menu features innovative preparations of the finest global seafood in an elegant setting."
  },
  {
    "name": "Katz's Delicatessen",
    "description": "An iconic New York institution since 1888, famous for its hand-carved pastrami and corned beef sandwiches. The bustling, no-frills atmosphere is part of its enduring charm."
  },
  {
    "name": "Eleven Madison Park",
    "description": "A three-Michelin-starred restaurant offering an innovative tasting menu focusing on locally sourced, plant-based ingredients. Known for its impeccable service and artistic presentation."
  },
  {
    "name": "Peter Luger Steak House",
    "description": "A Brooklyn institution since 1887, Peter Luger is famous for its dry-aged steaks and old-school, cash-only policy. The no-frills atmosphere focuses attention on the exceptional quality of the meat."
  },
  {
    "name": "Di Fara Pizza",
    "description": "A legendary Brooklyn pizzeria, Di Fara is known for its handcrafted pies made by founder Dom DeMarco. Each pizza is a work of art, featuring high-quality ingredients and meticulous preparation."
  },
  {
    "name": "Balthazar",
    "description": "A SoHo classic, Balthazar offers authentic French brasserie fare in a vibrant, bustling atmosphere. Known for its fresh seafood, classic French dishes, and popular weekend brunch."
  },
  {
    "name": "Momofuku Ko",
    "description": "Chef David Chang's two-Michelin-starred restaurant offers an ever-changing tasting menu blending Asian and Western influences. The intimate counter seating provides a unique, interactive dining experience."
  },
  {
    "name": "The Halal Guys",
    "description": "Starting as a food cart, The Halal Guys has become a New York institution. Famous for its chicken and gyro over rice, topped with their legendary white sauce. Now with brick-and-mortar locations."
  },
  {
    "name": "Russ & Daughters",
    "description": "A New York classic since 1914, specializing in traditional Jewish appetizing foods. Famous for its hand-sliced smoked salmon, bagels, and other Jewish delicacies. Now includes a sit-down cafe."
  },
  {
    "name": "Lombardi's",
    "description": "America's first pizzeria, established in 1905. Lombardi's continues to serve classic New York-style pizza from its coal-fired oven. Known for its simple, high-quality ingredients and historic charm."
  },
  {
    "name": "Joe's Shanghai",
    "description": "Famous for introducing soup dumplings to New York, Joe's Shanghai offers authentic Shanghai-style cuisine. The bustling, no-frills atmosphere adds to the authentic experience."
  }
]

# The Prompt

The following cell creates the prompt for our restaurant recommendation example.

- It uses a *request source* to obtain the location directly from the execution context.
- It incorporates the user's name into the prompt instructions from the user_info_fv

In [8]:
from tecton_gen_ai.utils.tecton_utils import make_request_source
from tecton_gen_ai.fco import prompt

#define real-time request source
location_request = make_request_source(location = str)

#create the prompt
@prompt(sources=[location_request, user_info_fv])  # specifies sources of data for the prompt
def sys_prompt(location_request, user_info_fv ):
    location = location_request["location"]
    name = user_info_fv["name"]
    return f"""
    Address the user by their name. Their name is {name}.
    You are a consierge service that recommends restaurants.
    Respond to the user's question. 
    If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
    Only suggest restaurants that are in or near {location}. 
    """


# The Knowledge

Allows you to add your unstructured data 
self select
retrieve data

In [12]:

from tecton_gen_ai.testing.utils import make_local_vector_db_config
from tecton_gen_ai.testing import make_local_source
from tecton_gen_ai.fco import source_as_knowledge

#provide a vector db config
conf = make_local_vector_db_config()

src = make_local_source(
    "restaurant_descriptions",
    restaurant_descriptions,
    ["name"],
    description="Restaurant descriptions",
)

#create embeddings of the restaurant descriptions in the vector DB
restaurant_knowledge = source_as_knowledge(
    src,
    description="Restaurant descriptions",
    vector_db_config=conf,
    vectorize_column="description",
)



# Agent Service

This example agent service, encapsulates the context and information retrieval being made available:
- sys_prompt 
- restaurant_knowledge
- user_info_fv & recent_eats_fv feature views

In a production application, an Agent Service is deployed to the Tecton Platform to provide prompts, knowledge and data retrieval through features as tools at scale, through Tecton's serving infrastructure.

In [13]:
from tecton_gen_ai.agent import AgentClient, AgentService

#define the Tecton Agent Service specifying the prompts, sources of knowledge and feature tools 
service = AgentService(
    name="restaurant_recommender",
    prompts=[sys_prompt],
    knowledge=[restaurant_knowledge],
    tools=[ user_info_fv, recent_eats_fv]
)

# Agent Client
In order to interact with the LLM, you can create an Agent Client that interacts with the Agent Service and provides the interface through which the LLM workflow can retrieve the prompts, knowledge and feature data retrieval.

The following cell shows an example of this using LangChain and OpenAI's gpt-4o model:

In [14]:
from langchain_openai import ChatOpenAI

#create an agent client that provides context for LLM workflows
client = AgentClient.from_local(service)

# instantiate LLM model for  LangChain 
langchain_llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

# create invocable agent for LangChain 
langchain_agent = client.make_agent(langchain_llm, system_prompt="sys_prompt")


# Tests

The following test uses the user's id and their current location to respond to the user request.

- It uses knowledge to determine which restaurant's descriptions include a romantic or intimate experience.
- It uses the user's information to refer to them by name.
- It uses the current location of the user to determine which restaurants are in that location.

In [15]:
#from tecton_gen_ai.testing.utils import print_md

with client.set_context({"user_id": "user1", "location":"Charlotte, NC"}):
    print(langchain_agent.invoke({"input":"recommend a romantic dinner"})["output"])

None
For a romantic dinner in Charlotte, I recommend **Villa Antonio**. This elegant Italian restaurant offers a charming atmosphere along with authentic cuisine. You can enjoy their homemade pasta, extensive wine selection, and attentive service, making it perfect for a special occasion.

Some suggested menu items include:
- Caprese Salad
- Lobster Ravioli
- Tiramisu for dessert

Enjoy your evening, Jim!


In the following example, for `user2` the LLM uses his name, and their food preference from the user info feature view.

It also figures out a restaurant that the user has not been to recently by retrieving that data from a tool.

In [16]:

with client.set_context({"user_id": "user2", "location":"Charlotte, NC"}):
    print(langchain_agent.invoke({"input":"Recommend a place I haven't been to before that serves my preference"})["output"])

None
John, I recommend you try **Di Fara Pizza**. It's a legendary Brooklyn pizzeria known for its handcrafted pies made by founder Dom DeMarco. Each pizza is a work of art, featuring high-quality ingredients and meticulous preparation. 

You might enjoy their classic Margherita pizza or the pepperoni pizza, which are both crowd favorites. It's a great spot if you're looking for authentic Italian pizza!


In this example, the user just refers to a specific chef.
The LLM uses the knowledge to find that chef's restaurant and determines that it is not near the user's location but still recommends an option near them which offers a similar cuisine.

In [17]:

with client.set_context({"user_id": "user3", "location":"Ballantyne, Charlotte, NC"}):
    print(langchain_agent.invoke({"input":"I want to eat with Chef Eric Ripert"})["output"])

None
While Chef Eric Ripert is known for his restaurant Le Bernardin, which specializes in exquisite seafood, it's located in New York City and not in Ballantyne, Charlotte. However, if you're looking for a great seafood experience locally, I recommend **Mama Ricotta's**. 

They offer a warm, family-friendly atmosphere and are known for their delicious hand-tossed pizzas and homemade pasta. Some suggested menu items include their signature chicken parmesan and a variety of fresh pasta dishes. Enjoy your meal, Jane!


In the following example the user is asking for food like Chef Eric Ripert's but less expensive.
The LLM uses the knowledge it has about Chef Eric Riperts restaurant to suggest a less expensive alternative.

In [18]:
with client.set_context({"user_id": "user3", "location":"New York"}):
    print(langchain_agent.invoke({"input":"I have never been here, what restaurant is most like the ones I visit"})["output"])

None
None
Jane, based on your recent visits to "Wan Fu" and "Ru San's," I recommend trying **"Wan Fu Quality Chinese Cuisine."** This upscale Chinese restaurant focuses on authentic flavors and high-quality ingredients, providing a refined dining experience. You can expect a menu featuring both classic and innovative Chinese dishes.

If you're in the mood for Japanese cuisine, you might also enjoy **"Ru San's,"** which is known for its extensive sushi menu and all-you-can-eat options in a casual atmosphere. 

Both places would complement your recent dining experiences well!
